In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn import model_selection
from sklearn import preprocessing
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,matthews_corrcoef

In [2]:
#funciones para realizar estudio de ablation
def potencia(c):
    """Calcula y devuelve el conjunto potencia del 
       conjunto c.
    """
    if len(c) == 0:
        return [[]]
    r = potencia(c[:-1])
    return r + [s + [c[-1]] for s in r]

def combinaciones(c, n):
    """Calcula y devuelve una lista con todas las
       combinaciones posibles que se pueden hacer
       con los elementos contenidos en c tomando n
       elementos a la vez.
    """
    return [s for s in potencia(c) if len(s) == n]


In [3]:
# obtener el train para saber cuales son las features más correlacionadas
for i in range(45):
    if i==0:
        train = pd.read_pickle('salida/nuevo4i/SICK/TRAIN_'+str(i+1)+'.csv_.pickle')
    else:
        try:
            temp = pd.read_pickle('salida/nuevo4i/SICK/TRAIN_'+str(i+1)+'.csv_.pickle')
            train=pd.concat([train,temp])
        except:
            pass

In [4]:
train=train.reset_index(drop=True)
train=train.drop(["verbT","verbH"],axis=1)
y_ = train['clases']
train = train.drop(['clases'],axis=1)

In [5]:
# SICK
y_train=[]
for i in y_:
    if i == "ENTAILMENT":
        y_train.append(1)
    else:
        y_train.append(0)

In [6]:
# Obtener TEST
for i in range(50):
    if i==0:
        test = pd.read_pickle('salida/nuevo4i/SICK/TEST_'+str(i+1)+'.csv_.pickle')
    else:
        try:
            temp = pd.read_pickle('salida/nuevo4i/SICK/TEST_'+str(i+1)+'.csv_.pickle')
            test=pd.concat([test,temp])
        except:
            pass

In [7]:
test = test.reset_index(drop=True)
test=test.drop(["verbT","verbH"],axis=1)

y_t = test['clases']
test = test.drop(['clases'],axis=1)

In [8]:
# SICK
y_test=[]
for i in y_t:
    if i == "ENTAILMENT":
        y_test.append(1)
    else:
        y_test.append(0)

In [9]:
features=train.columns

# new_data = {'feature' : [], 'accuracy' : [],'std_accuracy' : [], 'precision_w' : [],'std_precision_w' : [],
#             'recall_w' : [],'std_recall_w' : [],'f1_w' : [],'std_f1_w' : [],'matthews' : [],'std_matthews' : []}

model = LogisticRegression(penalty="l2", C=1)
kfold = model_selection.KFold(n_splits=10,shuffle=True)

In [10]:
columnas_c=['distancias', 'entropias', 'mutinf', 'mearts',
       'max_info', 'sumas', 'semantics', 'nlp_semantics', 
        'max_info_t',  'entail', 'contra', 'neutral',
       'list_incomp', 'entropia_relaciones',  
       'Jaro-Winkler_rit',  'overlap_ent']

In [11]:
#combinaciones(columnas_c,10)

In [12]:
com=combinaciones(columnas_c,10)
for c_ in com:
    print("Calculando",c_)
    feats=c_
    train_temp=train[feats]
    test_temp=test[feats]
    scaler = preprocessing.MaxAbsScaler().fit(train_temp)
    X_scaled = scaler.transform(train_temp)
    train_temp = X_scaled
    scaler = preprocessing.MaxAbsScaler().fit(test_temp)
    X_scaled = scaler.transform(test_temp)
    test_temp = X_scaled
    model.fit(train_temp,y_train)
    predictions = model.predict(test_temp)
    print(confusion_matrix(y_test, predictions,labels=[1,0]))
    print("acccuracy: ",round(accuracy_score(y_test,predictions),3))
    print("precision: ",round(precision_score(y_test,predictions),3))
    print("recall: ",round(recall_score(y_test,predictions),3))
    print("f1_score: ",round(f1_score(y_test,predictions),3))
    print("matthews_score: ",round(matthews_corrcoef(y_test,predictions),3))
# X_train_cv=np.concatenate((train_temp, test_temp), axis=0)
# Y_train_cv=np.concatenate((y_train, y_test), axis=0)
# cv_results = model_selection.cross_val_score(model, X_train_cv, Y_train_cv, cv=kfold, scoring='accuracy')
# print("Accuracy: ",round(cv_results.mean(),2),cv_results.std())
# cv_results = model_selection.cross_val_score(model, X_train_cv, Y_train_cv, cv=kfold, scoring='precision')
# print("Precision: ",round(cv_results.mean(),2),cv_results.std())
# cv_results = model_selection.cross_val_score(model, X_train_cv, Y_train_cv, cv=kfold, scoring='recall')
# print("Recall: ",round(cv_results.mean(),2),cv_results.std())
# cv_results = model_selection.cross_val_score(model, X_train_cv, Y_train_cv, cv=kfold, scoring='f1')
# print("F1 score: ",round(cv_results.mean(),2),cv_results.std())
# cv_results = model_selection.cross_val_score(model, X_train_cv, Y_train_cv, cv=kfold, scoring='matthews_corrcoef')
# print("Matthews: ",round(cv_results.mean(),2),cv_results.std())

Calculando ['distancias', 'entropias', 'mutinf', 'mearts', 'max_info', 'sumas', 'semantics', 'nlp_semantics', 'max_info_t', 'entail']


[[ 801  603]
 [ 627 2875]]
acccuracy:  0.749
precision:  0.561
recall:  0.571
f1_score:  0.566
matthews_score:  0.39
Calculando ['distancias', 'entropias', 'mutinf', 'mearts', 'max_info', 'sumas', 'semantics', 'nlp_semantics', 'max_info_t', 'contra']
[[ 742  662]
 [ 610 2892]]
acccuracy:  0.741
precision:  0.549
recall:  0.528
f1_score:  0.538
matthews_score:  0.358
Calculando ['distancias', 'entropias', 'mutinf', 'mearts', 'max_info', 'sumas', 'semantics', 'nlp_semantics', 'entail', 'contra']
[[1161  243]
 [1078 2424]]
acccuracy:  0.731
precision:  0.519
recall:  0.827
f1_score:  0.637
matthews_score:  0.471
Calculando ['distancias', 'entropias', 'mutinf', 'mearts', 'max_info', 'sumas', 'semantics', 'max_info_t', 'entail', 'contra']
[[ 801  603]
 [ 627 2875]]
acccuracy:  0.749
precision:  0.561
recall:  0.571
f1_score:  0.566
matthews_score:  0.39
Calculando ['distancias', 'entropias', 'mutinf', 'mearts', 'max_info', 'sumas', 'nlp_semantics', 'max_info_t', 'entail', 'contra']
[[ 800  